Analysis of Striker Finishing Quality Using FBref/Opta Data

Pull data and save

In [3]:
# Imports and download data from fbref
import lxml
import pandas as pd

url = 'https://fbref.com/en/comps/Big5/shooting/players/Big-5-European-Leagues-Stats'

df_url = pd.read_html(url)

df_url[0].head()


Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player             Nation                Pos   
0                  1         Max Aarons            eng ENG                 DF   
1                  2   Brenden Aaronson             us USA              MF,FW   
2                  3    Paxten Aaronson             us USA                 MF   
3                  4  Keyliane Abdallah             fr FRA                 FW   
4                  5   Yunis Abdelhamid             ma MAR                 DF   

  Unnamed: 4_level_0  Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Squad                Comp                Age   
0        Bournemouth  eng Premier League                 23   
1       Union Berlin       de Bundesliga                 22   
2     Eint Frankfurt       de Bundesliga                 19   
3          Marseille          fr Ligue 1                 17   
4              Reims          fr Ligue 1                 35   

  Unnamed: 7_level_0 Unnamed: 8_level_0 Standard  ...                    \
                Born                90s      Gls  ...  Dist FK PK PKatt   
0               2000               13.7        0  ...  23.9  0  0     0   
1               2000               14.1        2  ...  18.4  0  0     0   
2               2003                1.1        0  ...  15.1  0  0     0   
3               2006                0.0        0  ...   NaN  0  0     0   
4               1987               30.9        4  ...  15.0  0  1     1   

  Expected                            Unnamed: 26_level_0  
        xG npxG npxG/Sh  G-xG np:G-xG             Matches  
0      0.0  0.0    0.02   0.0     0.0             Matches  
1      2.0  2.0    0.11   0.0     0.0             Matches  
2      0.1  0.1    0.06  -0.1    -0.1             Matches  
3      0.0  0.0     NaN   0.0     0.0             Matches  
4      3.4  2.6    0.13  +0.6    +0.4             Matches  

[5 rows x 27 columns]